In [12]:
import hashlib
import os.path
import pandas as pd
from datetime import datetime, timedelta

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/calendar.events']
TIME_ZONE = 'America/Los_Angeles'

In [3]:
creds = None
if os.path.exists('creds/token.json'):
    print('json exists!')
    creds = Credentials.from_authorized_user_file('creds/token.json', SCOPES)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
                'creds/credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
    with open('creds/token.json', 'w') as token:
        token.write(creds.to_json())

json exists!


In [71]:
service = build('calendar', 'v3', credentials=creds)

# Call the Calendar API
now = datetime.utcnow().isoformat() + 'Z'  # 'Z' indicates UTC time
print('Getting the upcoming 10 events')
events_result = service.events().list(calendarId='primary', timeMin=now,
                                                    maxResults=10, singleEvents=True,
                                                    orderBy='startTime').execute()
events = events_result.get('items', [])

Getting the upcoming 10 events


In [76]:
'clandinin' in events[0]['id']

False

In [70]:
d = datetime.now().date()
tomorrow = datetime(d.year, d.month, d.day, 12) + timedelta(days=1)
start = tomorrow.isoformat()
end = (tomorrow + timedelta(hours=1)).isoformat()
body={
    "summary": 'Automating calendar',
    "description": 'This is a tutorial example of automating google calendar with python',
    #"colorId": "0",
    "start": {"dateTime": start, "timeZone": 'America/Los_Angeles'},
    "end": {"dateTime": end, "timeZone": 'America/Los_Angeles'},
    }
id_string = body["summary"] + body["start"]["dateTime"] + "created" + datetime.now().isoformat()
id = 'clandinin' + hashlib.sha224(id_string.encode()).hexdigest()
body["id"] = id
event_result = service.events().insert(calendarId='primary',
                                       body=body
                                    ).execute()